# Compositional flow simulation
In this tutorial, we provide a step-by-step guide to conducting a compositional flow simulation in porepy. A compositional model is designed to simulate the transport of multiple components within a multi-phase flow system. First, we will present the compositional flow equations for two-phase system-namingly, liquid (L) and vapour (V) comprising salt (NaCl) and water ($\text{H}_2\text{O}$) components. We will assume negligible capillary pressure, gravity segregation, chemcical reactions between components, and energy dispersion respectively. Subsequently, we will introduce further assumptions to simplify the model to represent a linear tracer-like setting, with which we will investigate its solution setup in porepy using basically the formulation in `GeothermalFlowModel`class.

All mathematical formulations of compositional model, including the different notations and their units used in this tutorial are taken from [Formulation for compositional flow in porous media with
gravity effects](https://www.overleaf.com/5672663691knrhbpmwdbfj#51d404 "Optional Title")


## Two-phase two-component problem statement:
Given the physical domain $\Omega \subset \mathbb{R}^2,$ along with constant fluid and rock properties data such as porosity, permeability tensor, and constitutive relations. Appropriate boundary and initial condition data are also given. The problem is to find two sets of variables, primary variables, which includes global pressure ($p$), enthalpy(or temperature), and overall molar composition of salt ($z_{NaCl}$) and water ($z_{\text{H}_2\text{O}}$) and the secondary variables. Ensuring that the following governing equations are satisfied:
1. Pressure equation is given by:
\begin{align*}
\frac{\partial (\phi\rho)}{\partial t} + \nabla \cdot (\textbf{m}) = 0
\end{align*}
2. Mass Conservation for Each Component:
For a two-component system of water (H₂O) and salt (NaCl), the conservative equation of water component in the mixture is given as follows:
\begin{align*}
\frac{\partial (\phi\rho z_{H_20})}{\partial t} + \nabla \cdot (f_{H_2O}(\textbf{m} - \Delta \hat{\rho}_{H_20})) = 0.
\end{align*}
At each time step, the salt concentration in the mixture can be computed using the component constraint given by:
\begin{align*}
z_{H_2O} + z_{NaCl} = 1.
\end{align*}
3. Energy equation of the mixture is given by:
\begin{align*}
\frac{\partial (\phi\rho u + (1 - \phi)\rho_s u_s)}{\partial t} + \nabla \cdot ( -(h_{L}(\textbf{m}_L) + h_{V}(\textbf{m}_V)  - \textbf{K}_e\nabla T )) = 0,
\end{align*}. 

### Linear tracer formulation:
For simplicity, we will assume constant global pressure and temperature with respect to time in the system. Consequently, the pressure and energy equations become negligible in the above model. This assumption leads to the presence of a single liquid (L) phase, eliminating the need to account for multiphase interactions. We will omit the correlation calculations used to determine other phase-behavior thermodynamic properties. In this case, we primarily consider a mass transport equation of a single passive tracer (the NaCl component) in a water medium.

### Import required modules to simulate linear tracer flow.
Here we imports necessary modules and classes from porepy and porepy.composite, sets up the geometry, boundary conditions, initial conditions, and the equations for the compositional flow model.

In [14]:
from __future__ import annotations
import numpy as np
import porepy as pp
import porepy.composite as ppc
from porepy.models.geometry import ModelGeometry
from porepy.models.compositional_flow import(
    BoundaryConditionsCF,
    CFModelMixin,
    InitialConditionsCF,
    PrimaryEquationsCF,
)
from porepy.examples.geothermal_flow_via_correlations.LinearTracerConstitutiveDescription import SecondaryEquations,LiquidLikeCorrelations

### Domain Geometry
we start by defining the geometry of the domain (with specification of fracture type, the boundary points, and grid cell type and sizes) on which we shall solve the tracer model in the porepy sense. This class extends the ModelGeometry class and is customized for a specific 2D benchmark scenario. It overrides several methods to tailor the geometry and meshing process to the requirements of this benchmark model.

In [4]:
class Benchmark2DC1(ModelGeometry):
    def set_fractures(self) -> None:
        """Sets the fractures specifically for the benchmark 2D case 1.
        """
        self._fractures = pp.applications.md_grids.fracture_sets.benchmark_2d_case_1()
    def grid_type(self) -> str:
        """Specifies the type of grid to be used as simplex.
        """
        return self.params.get("grid_type", "simplex")
    def meshing_arguments(self) -> dict:
        """Overrides the meshing_arguments method for the benchmark case, with finer cell size adjustments.
        """
        cell_size = self.solid.convert_units(0.1, "m")
        mesh_args: dict[str, float] = {"cell_size": cell_size}
        return mesh_args
    def get_inlet_outlet_sides(self, sd: pp.Grid | pp.BoundaryGrid) -> np.ndarray:
        """Determines the inlet and outlet sides of the grid or boundary grids.
        """
        if isinstance(sd, pp.Grid):
            x = sd.face_centers.T
        elif isinstance(sd, pp.BoundaryGrid):
            x = sd.cell_centers.T
        else:
            raise ValueError("Type not expected.")
        sides = self.domain_boundary_sides(sd)
        idx = sides.all_bf
        rc = 0.25  # Radius of the sphere for inlet
        xc = np.array([0.0, 0.0, 0.0])  # Center of the sphere for inlet
        logical = Benchmark2DC1.harvest_sphere_members(xc, rc, x[idx])
        inlet_facets = idx[logical]
        rc = 0.25  # Radius of the sphere for outlet
        xc = np.array([1.0, 1.0, 0.0])  # Center of the sphere for outlet
        logical = Benchmark2DC1.harvest_sphere_members(xc, rc, x[idx])
        outlet_facets = idx[logical]
        return inlet_facets, outlet_facets
    @staticmethod
    def harvest_sphere_members(xc, rc, x):
        """Identifies points within a sphere of a given radius and center
        """
        dx = x - xc
        r = np.linalg.norm(dx, axis=1)
        return np.where(r < rc, True, False)

### Boundary Conditions
The `BoundaryConditions` class extends the `BoundaryConditionsCF` class to define various boundary conditions for the compositional flow model, including pressure, enthalpy, and component fractions, etc, since the default boundary condition is zero Dirichlet.
This class includes methods for setting Dirichlet boundary conditions for different types of flux (Fourier, Darcy, and advective) based on inlet and outlet sides of the grid. Additionally, it provides methods to define boundary conditions for pressure, enthalpy, component fractions, and temperature.
To close the above system of linear equations, we consider the following set of boundary conditions for the unknown variables.



In [15]:
class BoundaryConditions(BoundaryConditionsCF):
    """See parent class how to set up BC. Default is all zero and Dirichlet."""
    def bc_type_fourier_flux(self, sd: pp.Grid) -> pp.BoundaryCondition:
        """Sets Dirichlet boundary conditions for Fourier flux based on inlet and outlet sides.
        """
        facet_idx = np.concatenate(self.get_inlet_outlet_sides(sd))
        return pp.BoundaryCondition(sd, facet_idx, "dir")

    def bc_type_darcy_flux(self, sd: pp.Grid) -> pp.BoundaryCondition:
        """Sets dirichlet boundary conditions for Darcy flux based on inlet and outlet sides.
        """
        facet_idx = np.concatenate(self.get_inlet_outlet_sides(sd))
        return pp.BoundaryCondition(sd, facet_idx, "dir")

    def bc_type_advective_flux(self, sd: pp.Grid) -> pp.BoundaryCondition:
        """Sets dirichlet boundary conditions for advective flux based on inlet and outlet sides.
        """
        facet_idx = np.concatenate(self.get_inlet_outlet_sides(sd))
        return pp.BoundaryCondition(sd, facet_idx, "dir")

    def bc_values_pressure(self, boundary_grid: pp.BoundaryGrid) -> np.ndarray:
        """Sets dirichlet boundary conditions for advective flux based on inlet and outlet sides.
        """
        inlet_idx, outlet_idx = self.get_inlet_outlet_sides(boundary_grid)
        p_inlet = 15.0e6
        p_outlet = 10.0e6
        p = p_inlet * np.ones(boundary_grid.num_cells) 
        p[inlet_idx] = p_inlet
        p[outlet_idx] = p_outlet
        return p

    def bc_values_enthalpy(self, boundary_grid: pp.BoundaryGrid) -> np.ndarray:
        """Sets enthalpy boundary conditions for the boundary grid
        """
        inlet_idx, _ = self.get_inlet_outlet_sides(boundary_grid)
        h_init = 2.0e6
        h_inlet = 2.0e6
        h = h_init * np.ones(boundary_grid.num_cells)
        h[inlet_idx] = h_inlet
        return h

    def bc_values_overall_fraction(
        self, component: ppc.Component, boundary_grid: pp.BoundaryGrid
    ) -> np.ndarray:
        """ Sets Dirichlet boundary conditions for overall composition at the boundary grid
        """
        inlet_idx, _ = self.get_inlet_outlet_sides(boundary_grid)
        z_init = 0.15
        z_inlet = 1.0
        if component.name == "H2O":
            z_H2O = (1 - z_init) * np.ones(boundary_grid.num_cells)
            z_H2O[inlet_idx] = 1 - z_inlet
            return z_H2O
        else:
            z_NaCl = z_init * np.ones(boundary_grid.num_cells)
            z_NaCl[inlet_idx] = z_inlet
            return z_NaCl

    def bc_values_temperature(self, boundary_grid: pp.BoundaryGrid) -> np.ndarray:
        """ Sets Dirichlet boundary conditions for temperature at the boundary grid
        """
        h = self.bc_values_enthalpy(boundary_grid)
        factor = 630.0 / 2.0e6
        T = factor * h
        return T

### Initial conditions
Here, we specify the the initial condition (at time $t = 0$) data for the unknow variables for all the grid cells as follows:

The `InitialConditions` class extends `InitialConditionsCF` to define the initial conditions for the different unknow variables in the compositional flow model.



In [6]:
class InitialConditions(InitialConditionsCF):
    """See parent class how to set up BC. Default is all zero and Dirichlet.
    """
    def initial_pressure(self, sd: pp.Grid) -> np.ndarray:
        """Sets the initial pressure distribution across the grid
        """
        p_init = 15.0e6
        p_outlet = 15.0e6
        xc = sd.cell_centers.T
        def p_D(xv):
            p_val = (1-xv[0]/2)*p_init + (xv[0]/2)*p_outlet
            return p_val
        p_vals = np.fromiter(map(p_D, xc),dtype=float)
        return p_vals

    def initial_enthalpy(self, sd: pp.Grid) -> np.ndarray:
        """Sets the initial condition data for enthalpy
        """
        h = 2.0e6
        return np.ones(sd.num_cells) * h

    def initial_overall_fraction(
        self, component: ppc.Component, sd: pp.Grid
    ) -> np.ndarray:
        """Sets the initial condition data for overall fraction of a given component 
        """
        z = 0.15
        if component.name == "H2O":
            return (1 - z) * np.ones(sd.num_cells)
        else:
            return z * np.ones(sd.num_cells)

    def initial_temperature(self, sd: pp.Grid) -> np.ndarray:
        """Computes the initial temperature based on the initial enthalpy
        """
        h = self.initial_enthalpy(sd)
        factor = 630.0 / 2.0e6
        T = factor * h
        return T

### Model equations setup

The `ModelEquations` class extends `PrimaryEquationsCF` and `SecondaryEquations` to collect primary flow and transport equations, and secondary equations which provide substitutions for independent saturations and partial fractions.

This class includes a method `set_equations` to call the equations. The parent classes don't use `super()`, so the user must provide the proper order resolution. This approach is consistent with other models, potentially due to the sparsity pattern.

The `set_equations` method first sets the flow and transport equations in the mixed-dimensional (MD) setting by calling `PrimaryEquationsCF.set_equations(self)`, and then handles the local elimination of dangling secondary variables by calling `SecondaryEquations.set_equations(self)`.

In [7]:

class ModelEquations(
    PrimaryEquationsCF,
    SecondaryEquations
):
    """Collecting primary flow and transport equations, and secondary equations
    which provide substitutions for independent saturations and partial fractions.
    """
    def set_equations(self):
        """Call to the equation. Parent classes don't use super(). User must provide
        proper order resultion.
        """
        # Flow and transport in MD setting
        PrimaryEquationsCF.set_equations(self)
        # local elimination of dangling secondary variables
        SecondaryEquations.set_equations(self)

### Define fluid mixture
The `FluidMixture` class extends `ppc.FluidMixtureMixin` to create a brine mixture with two components, H2O and NaCl. This class includes methods to set the components, configure the phase properties, and establish dependencies of phase properties. 

#### Key Methods and Attributes

- **enthalpy**: Callable attribute provided by `VariablesEnergyBalance`.

- **get_components**: Sets H2O as the reference component, eliminating \(z_{H2O}\). It loads species ["H2O", "NaCl"] and returns a sequence of components.

- **get_phase_configuration**: Configures the phase with liquid-like correlations for the components.

- **dependencies_of_phase_properties**: Establishes the dependencies of phase properties, including pressure, enthalpy, and the fraction of NaCl.

- **set_components_in_phases**: Applies the unified assumption by default, ensuring all components are present in all phases.



In [8]:
from typing import Callable,Sequence

class FluidMixture(ppc.FluidMixtureMixin):
    """Mixture mixin creating the brine mixture with two components."""

    enthalpy: Callable[[list[pp.Grid]], pp.ad.MixedDimensionalVariable]
    """Provided by :class:`~porepy.models.compositional_flow.VariablesEnergyBalance`."""

    def get_components(self) -> Sequence[ppc.Component]:
        """Setting H20 as first component in Sequence makes it the reference component.
        z_H20 will be eliminated.
        
        """
        species = ppc.load_species(["H2O", "NaCl"])
        components = [ppc.Component.from_species(s) for s in species]
        return components

    def get_phase_configuration(
        self, components: Sequence[ppc.Component]
    ) -> Sequence[tuple[ppc.AbstractEoS, int, str]]:
        """Configures the phase with liquid-like correlations for the components.
        """
        eos_L = LiquidLikeCorrelations(components)
        return [(eos_L, 0, "liq")]

    def dependencies_of_phase_properties(
        self, phase: ppc.Phase
    ) -> Sequence[Callable[[pp.GridLikeSequence], pp.ad.Operator]]:
        """Establishes the dependencies of phase properties, including pressure, enthalpy, and the fraction of NaCl.
        """
        z_NaCl = [
            comp.fraction
            for comp in self.fluid_mixture.components
            if comp != self.fluid_mixture.reference_component
        ]
        return [self.pressure, self.enthalpy] + z_NaCl

    def set_components_in_phases(
        self, components: Sequence[ppc.Component], phases: Sequence[ppc.Phase]
    ) -> None:
        """Applies the unified assumption by default, ensuring all components are present in all phases

        """
        super().set_components_in_phases(components, phases)


### Complete linear tracer model

The `LinearTracerFlowModel` class combines various components to create a model for linear tracer flow. This class extends `Benchmark2DC1`, `FluidMixture`, `InitialConditions`, `BoundaryConditions`, `ModelEquations`, and `CFModelMixin`.


In [9]:
class LinearTracerFlowModel(
    Benchmark2DC1,
    FluidMixture,
    InitialConditions,
    BoundaryConditions,
    ModelEquations,
    CFModelMixin,
):

    def relative_permeability(
        self, 
        saturation: pp.ad.Operator
    ) -> pp.ad.Operator:
        return saturation

### Constant solid fluid parameters data.
Here we sets up the constant parameters and time stepping configurations required for the linear tracer flow model simulation.

##### Time Management
The `time_manager` is configured using the `pp.TimeManager` class to handle the simulation schedule, time steps, and iteration settings.

- **day**: Number of seconds in a day, defined as \(86400\).
- **t_scale**: Time scaling factor, set to \(0.01\).
- **schedule**: Time points for the simulation, from \(0.0\) to \(10 \times \text{day} \times \text{t_scale}\).
- **dt_init**: Initial time step, set to \(1.0 \times \text{day} \times \text{t_scale}\).
- **constant_dt**: Whether the time step is constant, set to `True`.
- **iter_max**: Maximum number of iterations, set to `50`.
- **print_info**: Whether to print information during simulation, set to `True`.

##### Material Constants
The `solid_constants` and `material_constants` define the physical properties of the solid material used in the simulation.

- **permeability**: Set to $(9.869233 \times 10^{-14})$.
- **porosity**: Set to (0.2).
- **thermal_conductivity**: Set to (1.92).

#### Simulation Parameters
The `params` dictionary collects all the settings and configurations required for the simulation.
- **material_constants**: Contains the `solid_constants`.
- **eliminate_reference_phase**: Whether to eliminate the reference phase (`s_liq`), set to `True`.
- **eliminate_reference_component**: Whether to eliminate the reference component (`z_H2O`), set to `True`.
- **time_manager**: Contains the `time_manager` object.
- **prepare_simulation**: Whether to prepare the simulation, set to `False`.
- **reduce_linear_system_q**: Whether to reduce the linear system, set to `False`.
- **petsc_solver_q**: Whether to use the PETSc solver, set to `False`.
- **nl_convergence_tol**: Nonlinear convergence tolerance, set to $(1.0 \times 10^{-3})$.
- **max_iterations**: Maximum number of iterations, set to `25`.

In [10]:
day = 86400
t_scale = 0.01
time_manager = pp.TimeManager(
    schedule=[0.0, 10.0 * day * t_scale],
    dt_init=1.0 * day * t_scale,
    constant_dt=True,
    iter_max=50,
    print_info=True,
)
solid_constants = pp.SolidConstants(
    {"permeability": 9.869233e-14, "porosity": 0.2, "thermal_conductivity": 1.92}
)
material_constants = {"solid": solid_constants}
params = {
    "material_constants": material_constants,
    "eliminate_reference_phase": True,  # s_liq eliminated, default is True
    "eliminate_reference_component": True,  # z_H2O eliminated, default is True
    "time_manager": time_manager,
    "prepare_simulation": False,
    "reduce_linear_system_q": False,
    "petsc_solver_q": False,
    "nl_convergence_tol": 1.0e-3,
    "max_iterations": 25,
}

### Run complete simulation


In [11]:
model = LinearTracerFlowModel(params)
model.prepare_simulation()
model.exporter.write_vtu()
pp.run_time_dependent_model(model,params)
print("Total number of DoF: ", model.equation_system.num_dofs())
print("Mixed-dimensional grid information: ", model.mdg)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Total number of DoF:  2904
Mixed-dimensional grid information:  Mixed-dimensional grid. 
Maximum dimension present: 2 
Minimum dimension present: 0 
Size of highest dimensional grid: Cells: 368. Nodes: 261
In lower dimensions: 
6 grids of dimension 1, with in total 46 cells and 64 nodes. 
9 grids of dimension 0, with in total 9 cells and 0 nodes. 
Total number of interfaces: 24
6 interfaces between grids of dimension 2 and 1 with in total 92 cells.
18 interfaces between grids of dimension 1 and 0 with in total 30 cells.



Visualization of the solution to this problem is demonstrated in the GIF below. ParaView for the visualization is used to visualize the transport of a tracer in the medium. 

<img src='img/linear_tracer_transport.gif'  width=600>